In [ ]:
import pandas as pd
from scipy.sparse import vstack, hstack
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# read yelp.csv into a DataFrame using a relative path
import pandas as pd

# mount google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/BT4222/data/MBTI_features.csv'
mbti = pd.read_csv(path)

## Hyper parameter tuning


In [ ]:
path = '/content/drive/My Drive/BT4222/data/combined_with_split_tags.csv'
mbti = pd.read_csv(path)

In [ ]:
mbti.head()

,Tag,Cleaned_Text,Length,Emoticons_count,Emoticons Avg,Unique_Words,TTR,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Afinn Score,Polarity,Subjectivity,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject,lowercase,uppercase,uppercase_num,proper cap,contractions_num,emotionalpunctuations_num,readable_num,hedge_perc,firstperson_perc,thirdperson_perc,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,characters_per_word,syll_per_word,words_per_sentence,sentences_per_paragraph,type_token_ratio,characters,syllables,words,wordtypes,sentences,paragraphs,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,pronoun,preposition,nominalization,pronoun.1,interrogative,article,subordination,conjunction.1,preposition.1,count,E_I,N_S,F_T,J_P
0,ENFP,disagree think theory given validity rather be...,1682,0,0.0,25,0.014863,0.002378,0.003567,0.000595,0.005945,0.002378,0.007134,0.013674,0.002973,0.000595,0.008918,3,0.059524,0.436508,124,42,61,53,23,85,34,0,0,0,15,1,0.00,3,486,0.438272,0.014403,0.024691,32.261518,39.660000,11.086820,3.580430,37.599575,101.642251,22.104973,18.166667,12.927531,4.636943,1.460722,78.500000,6.0,0.515924,2184,688,471,243,6,1,109,73,161,23,6,25,36,67,23,3,0,1,0,0,0,228,E,N,F,P
1,INTP,selfish generally perform task better person a...,1265,0,0.0,26,0.020553,0.010277,0.006324,0.010277,0.001581,0.002372,0.012648,0.011858,0.003953,0.000791,0.005534,5,-0.009722,0.660648,73,25,46,30,53,92,27,0,0,0,25,1,0.02,1,401,0.443890,0.089776,0.039900,38.985804,48.357195,8.449873,-5.890321,44.289526,118.205112,20.748239,18.000000,12.389050,4.174564,1.311721,100.250000,4.0,0.436409,1674,526,401,175,4,1,72,42,96,18,2,18,60,51,7,3,0,0,0,0,0,177,I,N,T,P
2,INFP,personality system technically separate types...,738,0,0.0,25,0.033875,0.001355,0.000000,0.004065,0.001355,0.000000,0.001355,0.006775,0.000000,0.000000,0.002710,12,0.121053,0.382310,55,26,31,33,16,34,18,0,0,0,4,1,0.02,0,269,0.498141,0.040892,0.048327,102.699182,131.734907,7.390028,-162.121933,110.871375,280.524164,28.690465,31.000000,21.792209,3.962825,1.133829,269.000000,1.0,0.531599,1066,305,269,143,1,1,31,22,82,16,3,16,21,28,3,0,0,1,0,0,0,129,I,N,F,P
3,INTP,like functions general theyre basically ways p...,1550,0,0.0,25,0.016129,0.002581,0.006452,0.002581,0.003871,0.007097,0.005806,0.018065,0.003871,0.003871,0.014194,10,0.108208,0.371320,80,35,55,33,33,82,25,0,0,0,3,1,0.01,2,389,0.519280,0.056555,0.051414,18.923333,22.510195,11.457340,36.177222,23.945475,69.142039,17.719601,11.222222,11.065275,4.752577,1.500000,43.111111,9.0,0.536082,1844,582,388,208,9,1,101,65,130,11,2,24,35,51,10,0,1,0,0,0,0,206,I,N,T,P
4,ENTP,even referring nihilism yeah point dying livin...,767,0,0.0,25,0.032595,0.011734,0.007823,0.006519,0.019557,0.005215,0.026076,0.013038,0.019557,0.002608,0.006519,-31,0.006090,0.518506,42,17,25,17,19,47,26,0,0,0,2,1,0.04,3,231,0.445887,0.004329,0.047619,90.284416,114.418831,9.474105,-140.796234,96.382684,249.181818,29.267851,42.000000,19.058689,4.320346,1.337662,231.000000,1.0,0.632035,998,309,231,146,1,1,42,23,58,13,6,12,21,30,3,1,0,0,0,0,0,110,E,N,T,P


In [ ]:
Features = mbti.drop(columns = ['count','Tag','E_I', 'N_S', 'F_T', 'J_P'])
features = list(Features.columns.values)
features

['Cleaned_Text',
 'Length',
 'Emoticons_count',
 'Emoticons Avg',
 'Unique_Words',
 'TTR',
 'anger',
 'anticipation',
 'disgust',
 'fear',
 'joy',
 'negative',
 'positive',
 'sadness',
 'surprise',
 'trust',
 'Afinn Score',
 'Polarity',
 'Subjectivity',
 'num_noun',
 'num_adj',
 'num_prep',
 'num_det',
 'num_pron',
 'num_verb',
 'num_adverb',
 'num_interject',
 'lowercase',
 'uppercase',
 'uppercase_num',
 'proper cap',
 'contractions_num',
 'emotionalpunctuations_num',
 'readable_num',
 'hedge_perc',
 'firstperson_perc',
 'thirdperson_perc',
 'Kincaid',
 'ARI',
 'Coleman-Liau',
 'FleschReadingEase',
 'GunningFogIndex',
 'LIX',
 'SMOGIndex',
 'RIX',
 'DaleChallIndex',
 'characters_per_word',
 'syll_per_word',
 'words_per_sentence',
 'sentences_per_paragraph',
 'type_token_ratio',
 'characters',
 'syllables',
 'words',
 'wordtypes',
 'sentences',
 'paragraphs',
 'long_words',
 'complex_words',
 'complex_words_dc',
 'tobeverb',
 'auxverb',
 'conjunction',
 'pronoun',
 'preposition',
 'no

In [ ]:
X = mbti.iloc[:,:-4]
X = X.drop(columns = [ 'Tag','count'])
y_targets = mbti.iloc[:,-4:]
X

,Cleaned_Text,Length,Emoticons_count,Emoticons Avg,Unique_Words,TTR,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,Afinn Score,Polarity,Subjectivity,num_noun,num_adj,num_prep,num_det,num_pron,num_verb,num_adverb,num_interject,lowercase,uppercase,uppercase_num,proper cap,contractions_num,emotionalpunctuations_num,readable_num,hedge_perc,firstperson_perc,thirdperson_perc,Kincaid,ARI,Coleman-Liau,FleschReadingEase,GunningFogIndex,LIX,SMOGIndex,RIX,DaleChallIndex,characters_per_word,syll_per_word,words_per_sentence,sentences_per_paragraph,type_token_ratio,characters,syllables,words,wordtypes,sentences,paragraphs,long_words,complex_words,complex_words_dc,tobeverb,auxverb,conjunction,pronoun,preposition,nominalization,pronoun.1,interrogative,article,subordination,conjunction.1,preposition.1
0,disagree think theory given validity rather be...,1682,0,0.000000,25,0.014863,0.002378,0.003567,0.000595,0.005945,0.002378,0.007134,0.013674,0.002973,0.000595,0.008918,3,0.059524,0.436508,124,42,61,53,23,85,34,0,0,0,15,1,0.00,3,486,0.438272,0.014403,0.024691,32.261518,39.660000,11.086820,3.580430,37.599575,101.642251,22.104973,18.166667,12.927531,4.636943,1.460722,78.500000,6.0,0.515924,2184,688,471,243,6,1,109,73,161,23,6,25,36,67,23,3,0,1,0,0,0
1,selfish generally perform task better person a...,1265,0,0.000000,26,0.020553,0.010277,0.006324,0.010277,0.001581,0.002372,0.012648,0.011858,0.003953,0.000791,0.005534,5,-0.009722,0.660648,73,25,46,30,53,92,27,0,0,0,25,1,0.02,1,401,0.443890,0.089776,0.039900,38.985804,48.357195,8.449873,-5.890321,44.289526,118.205112,20.748239,18.000000,12.389050,4.174564,1.311721,100.250000,4.0,0.436409,1674,526,401,175,4,1,72,42,96,18,2,18,60,51,7,3,0,0,0,0,0
2,personality system technically separate types...,738,0,0.000000,25,0.033875,0.001355,0.000000,0.004065,0.001355,0.000000,0.001355,0.006775,0.000000,0.000000,0.002710,12,0.121053,0.382310,55,26,31,33,16,34,18,0,0,0,4,1,0.02,0,269,0.498141,0.040892,0.048327,102.699182,131.734907,7.390028,-162.121933,110.871375,280.524164,28.690465,31.000000,21.792209,3.962825,1.133829,269.000000,1.0,0.531599,1066,305,269,143,1,1,31,22,82,16,3,16,21,28,3,0,0,1,0,0,0
3,like functions general theyre basically ways p...,1550,0,0.000000,25,0.016129,0.002581,0.006452,0.002581,0.003871,0.007097,0.005806,0.018065,0.003871,0.003871,0.014194,10,0.108208,0.371320,80,35,55,33,33,82,25,0,0,0,3,1,0.01,2,389,0.519280,0.056555,0.051414,18.923333,22.510195,11.457340,36.177222,23.945475,69.142039,17.719601,11.222222,11.065275,4.752577,1.500000,43.111111,9.0,0.536082,1844,582,388,208,9,1,101,65,130,11,2,24,35,51,10,0,1,0,0,0,0
4,even referring nihilism yeah point dying livin...,767,0,0.000000,25,0.032595,0.011734,0.007823,0.006519,0.019557,0.005215,0.026076,0.013038,0.019557,0.002608,0.006519,-31,0.006090,0.518506,42,17,25,17,19,47,26,0,0,0,2,1,0.04,3,231,0.445887,0.004329,0.047619,90.284416,114.418831,9.474105,-140.796234,96.382684,249.181818,29.267851,42.000000,19.058689,4.320346,1.337662,231.000000,1.0,0.632035,998,309,231,146,1,1,42,23,58,13,6,12,21,30,3,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,kinda amusing deal trust relate usually someon...,599,0,0.000000,24,0.040067,0.000000,0.005008,0.001669,0.000000,0.005008,0.003339,0.006678,0.003339,0.001669,0.005008,6,0.150000,0.594444,46,13,10,11,26,51,18,0,0,0,9,0,0.06,10,196,0.515306,0.137755,0.035714,74.877551,94.352653,6.247678,-92.675408,81.053061,207.224490,22.748418,22.000000,17.708406,3.775510,1.188776,196.000000,1.0,0.668367,740,233,196,131,1,1,22,13,54,4,3,7,36,20,0,0,0,0,0,0,0
12822,social 9 description fit better probably descr...,617,23,0.037277,25,0.040519,0.000000,0.003241,0.003241,0.000000,0.006483,0.006483,0.011345,0.000000,0.001621,0.004

In [ ]:
from scipy.sparse import coo_matrix

X_train, X_test, y_train_targets, y_test_targets = train_test_split(X, y_targets,random_state=1, test_size = 0.2)
vect = CountVectorizer(max_features = 5000)
X_train_text = X_train['Cleaned_Text']
X_train_dtm = vect.fit_transform(X_train_text)
X_train_features = coo_matrix(X_train.iloc[:,1:])
X_train_dtm_features = hstack((X_train_dtm,X_train_features))
X_test_dtm = vect.transform(X_test['Cleaned_Text'])
X_test_features = coo_matrix(X_test.iloc[:,1:])
X_test_dtm_features = hstack((X_test_dtm,X_test_features))

## Grid search E_I

In [ ]:
GaussianNB().get_params().keys()

dict_keys(['priors', 'var_smoothing'])

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

gnb_pipeline = Pipeline([
    ('clf', GaussianNB()),
])
nb = gnb_pipeline.fit(X_train_dtm_features.todense(), y_train_targets['E_I'])
# alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# p_grid_NB = {'alpha': alphas, 'fit_prior' : [True, False], 'class_prior' : [None, [.1,.9],[.2, .8]]}
grid_params = {
    
  'clf__priors': [None],
  'clf__var_smoothing': np.logspace(0,-9, num=100)
}
clf1 = GridSearchCV(nb,  param_grid=grid_params)
clf1.fit(X_train_dtm_features.todense(), y_train_targets['E_I'])
print("Best Score: ", clf1.best_score_)
print("Best Params: ", clf1.best_params_)

Best Score:  0.713840155945419
Best Params:  {'clf__priors': None, 'clf__var_smoothing': 0.2848035868435802}


## Grid Search N_S

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

gnb_pipeline = Pipeline([
    ('clf', GaussianNB()),
])
nb = gnb_pipeline.fit(X_train_dtm_features.todense(), y_train_targets['N_S'])
grid_params = {
    
  'clf__priors': [None],
  'clf__var_smoothing': np.logspace(0,-9, num=100)
}
clf2 = GridSearchCV(nb,  param_grid=grid_params)
clf2.fit(X_train_dtm_features.todense(), y_train_targets['N_S'])
print("Best Score: ", clf2.best_score_)
print("Best Params: ", clf2.best_params_)

Best Score:  0.8205653021442496
Best Params:  {'clf__priors': None, 'clf__var_smoothing': 1.0}


## Grid Search F_T

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

gnb_pipeline = Pipeline([
    ('clf', GaussianNB()),
])
nb = gnb_pipeline.fit(X_train_dtm_features.todense(), y_train_targets['F_T'])
# alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# p_grid_NB = {'alpha': alphas, 'fit_prior' : [True, False], 'class_prior' : [None, [.1,.9],[.2, .8]]}
grid_params = {
    
  'clf__priors': [None],
  'clf__var_smoothing': np.logspace(0,-9, num=100)
}
clf3 = GridSearchCV(nb,  param_grid=grid_params)
clf3.fit(X_train_dtm_features.todense(), y_train_targets['F_T'])
print("Best Score: ", clf3.best_score_)
print("Best Params: ", clf3.best_params_)

Best Score:  0.6792397660818714
Best Params:  {'clf__priors': None, 'clf__var_smoothing': 3.5111917342151273e-09}


## Grid Search J_P

In [ ]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

gnb_pipeline = Pipeline([
    ('clf', GaussianNB()),
])
nb = gnb_pipeline.fit(X_train_dtm_features.todense(), y_train_targets['J_P'])
# alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# p_grid_NB = {'alpha': alphas, 'fit_prior' : [True, False], 'class_prior' : [None, [.1,.9],[.2, .8]]}
grid_params = {
    
  'clf__priors': [None],
  'clf__var_smoothing': np.logspace(0,-9, num=100)
}
clf4 = GridSearchCV(nb,  param_grid=grid_params)
clf4.fit(X_train_dtm_features.todense(), y_train_targets['J_P'])
print("Best Score: ", clf4.best_score_)
print("Best Params: ", clf4.best_params_)

Best Score:  0.5854775828460038
Best Params:  {'clf__priors': None, 'clf__var_smoothing': 0.08111308307896872}
